In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 

##### import warnings
warnings.filterwarnings(action = 'ignore')

In [7]:
path = 'C:\\Users\\cowal\\archive\\movies\\'

In [8]:
movie = pd.read_csv(path + 'ratings.csv', low_memory=False)
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [9]:
movie.shape

(100004, 4)

In [10]:
movie = movie.sort_values(by='timestamp', ascending=True).reset_index(drop =True)
movie.head(10)

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,21,5.0,828212412
4,409,25,4.0,828212412
5,409,30,5.0,828212412
6,409,32,3.0,828212412
7,409,35,4.0,828212412
8,409,85,5.0,828212412
9,409,16,4.0,828212412


In [11]:
meta = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [12]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [18]:
meta = meta.rename(columns={'id':'movieId'})
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] =meta['movieId'].astype(str)
movie = pd.merge(movie,meta[['movieId', 'original_title']], how='left', on='movieId')


In [43]:
movie = movie[movie['original_title'].notnull()].reset_index(drop= True)
movie

,userId,movieId,rating,timestamp,original_title
0,383,21,3.0,789652009,The Endless Summer
1,409,21,5.0,828212412,The Endless Summer
2,409,25,4.0,828212412,Jarhead
3,409,30,5.0,828212412,彼女の想いで
4,409,35,4.0,828212412,The Simpsons Movie
...,...,...,...,...,...
44989,251,40819,4.0,1476555871,Best Worst Movie
44990,547,47493,3.0,1476587644,To Kill a Priest
44991,251,1265,4.0,1476622980,Bridge to Terabithia
44992,251,44191,3.0,1476623070,Loose Screws


In [46]:
agg =movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head(10)

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."
6,"[Knockin' on Heaven's Door, The Grapes of Wrat..."
7,"[La passion de Jeanne d'Arc, Donnie Darko, Ita..."
8,"[Blackmail, Parenthood, Rosemary's Baby, Boogi..."
9,"[Young and Innocent, The Dark, Terminator Salv..."


In [47]:
movie['original_title'].unique()

array(['The Endless Summer', 'Jarhead', '彼女の想いで', ...,
       'The Lonedale Operator', 'Violeta se fue a los cielos',
       'To Kill a Priest'], dtype=object)

In [48]:
sentence = [] 
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))
    

In [54]:
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, vector_size=20, window =5, 
                           min_count=1, workers=4, batch_words=200, sg=1)

In [55]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)


[('Der bewegte Mann', 0.9957095384597778),
 ("L'Enfant sauvage", 0.9954582452774048),
 ('Auf der anderen Seite', 0.9950810670852661),
 ('Leaving Las Vegas', 0.9950066804885864),
 ('Tanguy', 0.9950057864189148),
 ('Planet of the Apes', 0.9945598244667053),
 ('Kafka', 0.9941155314445496),
 ('Falstaff', 0.9937899708747864),
 ('La Boum', 0.9937012195587158),
 ('Winnetou II', 0.9934428930282593)]